In [5]:
!pip install -q transformers==4.52.4
!pip install -q bitsandbytes==0.46.0
!pip install -q accelerate==1.7.0
!pip install -q langchain==0.3.25
!pip install -q langchainhub==0.1.21
!pip install -q langchain-chroma==0.2.4
!pip install -q langchain_experimental==0.3.4
!pip install -q langchain-community==0.3.24
!pip install -q langchain_huggingface==0.2.0
!pip install -q python-dotenv==1.1.0
!pip install -q pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 104.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### Import necessary libraris

In [35]:
import torch
# Thư viện học sâu phổ biến, dùng cho xử lý tensor và tăng tốc bằng GPU.
# Ví dụ: tensor = torch.tensor([1.0, 2.0]).to("cuda")

from transformers import BitsAndBytesConfig
# Cấu hình lượng tử hóa (quantization) mô hình giúp giảm bộ nhớ khi load mô hình lớn.
"""
Các tham số:
-----------
bnb_4bit_compute_dtype : torch.dtype
    Kiểu dữ liệu được sử dụng trong quá trình tính toán sau khi mô hình đã được lượng tử hóa.
    Ở đây sử dụng torch.float16 để cân bằng giữa hiệu suất và độ chính xác.

bnb_4bit_quant_type : str
    Phương pháp lượng tử hóa 4-bit. 'nf4' (Normal Float 4) là kỹ thuật lượng tử hóa tiên tiến giúp mô hình giữ được độ chính xác tốt hơn so với các kỹ thuật cũ như 'fp4'.

bnb_4bit_use_double_quant : bool
    Nếu đặt là True, sẽ sử dụng double quantization – nghĩa là dữ liệu đầu tiên được lượng tử hóa sang 8-bit trước, sau đó tiếp tục lượng tử hóa sang 4-bit.
    Điều này giúp giảm hơn nữa kích thước mô hình mà vẫn giữ được độ chính xác.

load_in_4bit : bool
    Nếu đặt là True, mô hình sẽ được tải vào ở định dạng 4-bit, giúp giảm tiêu thụ bộ nhớ đáng kể khi làm việc với các mô hình lớn như LLaMA, GPT, v.v.
"""
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
# - AutoTokenizer: tự động tải tokenizer tương ứng với mô hình.
# - AutoModelForCausalLM: tải mô hình ngôn ngữ dạng sinh (GPT-like).
# - pipeline: tạo pipeline inference nhanh cho các task NLP như text-generation, QA...
# Ví dụ:
# tokenizer = AutoTokenizer.from_pretrained("gpt2")
# model = AutoModelForCausalLM.from_pretrained("gpt2")
# text_gen = pipeline("text-generation", model=model, tokenizer=tokenizer)
# Dùng pipeline: khi bạn cần nhanh, demo, ít kiểm soát → tiện lợi, rất thích hợp cho LangChain.
# Gọi tokenizer + model: khi bạn muốn kiểm soát chi tiết, dùng cho R&D hoặc production nghiêm túc.

"""
LangChain là một framework mã nguồn mở giúp xây dựng các ứng dụng dựa trên mô
hình ngôn ngữ lớn (LLMs) — như chatbot, hệ thống hỏi đáp (RAG), tóm tắt, tác
nhân (agent), và nhiều hơn nữa.

Nó kết nối LLM với dữ liệu ngoài và công cụ thực tế, giúp mô hình "thông minh
hơn" và "hữu dụng hơn" thay vì chỉ sinh văn bản tĩnh.
"""

from langchain_huggingface import HuggingFaceEmbeddings
# Tạo hàm embedding từ mô hình trên HuggingFace, dùng trong các vector database.
# Ví dụ:
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

from langchain_huggingface.llms import HuggingFacePipeline
# Đóng gói HuggingFace pipeline (text-gen) thành một LangChain LLM để xây chain.
# Ví dụ:
# llm = HuggingFacePipeline(pipeline=text_gen)

from langchain.memory import ConversationBufferMemory
# Lưu lại toàn bộ cuộc hội thoại để sử dụng trong các bước tiếp theo.
# Ví dụ:
# memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

from langchain_community.chat_message_histories import ChatMessageHistory
# Quản lý lịch sử tin nhắn người dùng và hệ thống theo định dạng LangChain.
# Ví dụ:
# history = ChatMessageHistory()
# history.add_user_message("Xin chào"); history.add_ai_message("Chào bạn!")

from langchain_community.document_loaders import PyPDFLoader, TextLoader
# Dùng để đọc file PDF và file văn bản thuần thành các tài liệu LangChain.
# Ví dụ:
# docs = PyPDFLoader("tài_liệu.pdf").load()
# txt_docs = TextLoader("tài_liệu.txt").load()

from langchain.chains import ConversationalRetrievalChain
# Tạo chuỗi hỏi đáp có sử dụng retriever (vector search) và memory hội thoại.
# Ví dụ:
# qa_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=vectorstore.as_retriever(), memory=memory)

from langchain_experimental.text_splitter import SemanticChunker
# Chia nhỏ văn bản theo ngữ nghĩa (semantic), dựa vào embedding thay vì chỉ đếm ký tự.
# Ví dụ:
# chunker = SemanticChunker(embedding_model=embeddings)
# chunks = chunker.split_documents(docs)

from langchain_chroma import Chroma
# Vector store Chroma – lưu trữ và tìm kiếm vector đã được embedding.
# Ví dụ:
# db = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)

from langchain_text_splitters import RecursiveCharacterTextSplitter
# Cắt văn bản thành các đoạn nhỏ, có thể chồng lặp (overlap), đảm bảo giữ ngữ cảnh.
# Ví dụ:
# splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
# small_docs = splitter.split_documents(docs)

from langchain_core.runnables import RunnablePassthrough
# Một bước trung gian truyền nguyên dữ liệu không thay đổi – dùng để debug pipeline.
# Ví dụ:
# passthrough = RunnablePassthrough()
# result = passthrough.invoke("test")  # -> "test"

from langchain_core.output_parsers import StrOutputParser
# Dùng để trích xuất đầu ra dạng chuỗi từ LLM.
# Ví dụ:
# parser = StrOutputParser()
# output = parser.invoke({"text": "Xin chào"})  # -> "Xin chào"

from langchain import hub
# Truy cập các chain mẫu được chia sẻ trên LangChain Hub như RAG, chatbot, v.v.
# Ví dụ:
# chain = hub.pull("rlm/rag-prompt")


### Start here

#### Load file documents

In [36]:
# !gdown 1Wuq0COKnU9mCfMvTEq54DBLgAh3yYDx


In [37]:
Loader = PyPDFLoader
PATH = "/content/YOLOv10_Tutorials.pdf"
loader = Loader(PATH)
document = loader.load()

#### Embedding documents into vectors

In [38]:
embedding = HuggingFaceEmbeddings(
    model_name='bkai-foundation-models/vietnamese-bi-encoder'
    # intfloat/multilingual-e5-base
)

#### Split text by semantic

In [39]:
semantic_splitter = SemanticChunker(
    embeddings = embedding,
    breakpoint_threshold_type='percentile',
    breakpoint_threshold_amount=95,
    min_chunk_size=500,
    add_start_index = True
)

In [40]:
docs = semantic_splitter.split_documents(document)

In [41]:
len(docs)

32

In [42]:
type(docs)

list

#### Create vector database

In [43]:
vector_db = Chroma.from_documents(
    documents=docs,
    embedding=embedding
)

In [44]:
retriver = vector_db.as_retriever()
prompt = 'YOLOv10 dùng để làm gì ?'
results = retriver.invoke(prompt)

In [45]:
results[0].page_content

'AI VIETNAM (AIO2024) aivietnam.edu.vn\nHình 2: Hiệu suất của YOLOv10 khi so sánh với các mô hình khác. Trên tập dữ liệu COCO,\nYOLOv10 đạt được kết quả tốt nhất về khía cạnh Độ trễ (Latency) và Số lượng tham số mô\nhình (Number of parameters) trong khi vẫn giữ được độ chính xác (COCO AP) cao. Ảnh: [10]. Trong bài viết này, chúng ta sẽ cùng nhau tìm hiểu về YOLOv10 và cách sử dụng mô hình này. Thông qua đó, nhóm cũng sẽ trình bày sơ lược về bài toán Object Detection cũng như tóm tắt\nngắn gọn các phiên bản YOLO trước đó để bạn đọc có một cái nhìn tổng quan hơn về nội dung\nnày. Theo đó, bài viết được bố cục như sau:\n- Phần I:Giới thiệu về nội dung bài viết. - Phần II:Tóm tắt về bài toán Object Detection và các phiên bản YOLO đời trước. - Phần III:Trình bày nội dung YOLOv10. - Phần IV:Hướng dẫn cách cài đặt, huấn luyện và sử dụng YOLOv10. - Phần V:Trích dẫn tài liệu. 2'

#### Use quantization for model vicuna

In [46]:
MODEL_NAME = 'lmsys/vicuna-7b-v1.5'

# Define the offload folder
# offload_folder = "./offload"
config_nf4 = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config = config_nf4,
    device_map = 'auto',
    low_cpu_mem_usage=True,
    # offload_folder=offload_folder # Add offload folder
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model_pipeline = pipeline(
    "text-generation",
    model = model,
    tokenizer = tokenizer,
    max_new_tokens = 512,
    pad_token_id = tokenizer.eos_token_id,
    device_map ='auto'
)

llm = HuggingFacePipeline(
    pipeline = model_pipeline
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Device set to use cuda:0


#### Prompting

In [47]:
template_promt = hub.pull('rlm/rag-prompt')

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [48]:
print(template_promt)

input_variables=['context', 'question'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


In [49]:
def format_docs(docs):
  return '\n\n'.join(doc.page_content for doc in docs)

In [50]:
rag_chain = (
    {'context' : retriver | format_docs, 'question' : RunnablePassthrough()} # Changed 'content' to 'context'
    | template_promt
    | llm
    | StrOutputParser()
)

In [51]:
EXAMPLE_QUESTION = "YOLOv10 là gì"
output = rag_chain.invoke(EXAMPLE_QUESTION)

In [52]:
print(output)

Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: YOLOv10 là gì 
Context: AI VIETNAM (AIO2024) aivietnam.edu.vn
Hình 2: Hiệu suất của YOLOv10 khi so sánh với các mô hình khác. Trên tập dữ liệu COCO,
YOLOv10 đạt được kết quả tốt nhất về khía cạnh Độ trễ (Latency) và Số lượng tham số mô
hình (Number of parameters) trong khi vẫn giữ được độ chính xác (COCO AP) cao. Ảnh: [10]. Trong bài viết này, chúng ta sẽ cùng nhau tìm hiểu về YOLOv10 và cách sử dụng mô hình này. Thông qua đó, nhóm cũng sẽ trình bày sơ lược về bài toán Object Detection cũng như tóm tắt
ngắn gọn các phiên bản YOLO trước đó để bạn đọc có một cái nhìn tổng quan hơn về nội dung
này. Theo đó, bài viết được bố cục như sau:
- Phần I:Giới thiệu về nội dung bài viết. - Phần II:Tóm tắt về bài toán Object Detection và các phiên b

#### Streamlit deployment

| Mục        | Giải thích                                                             |
| ---------- | ---------------------------------------------------------------------- |
| Mục đích   | Nhận một URL công khai để truy cập Streamlit đang chạy trong notebook  |
| `curl`     | Lấy dữ liệu HTTP từ `local.lt`                                         |
| `local.lt` | Dịch vụ tạo tunnel, giúp truy cập cổng 8501 (Streamlit) từ bên ngoài   |
| Sử dụng    | Khi chạy Streamlit trong môi trường không thể mở cổng như Google Colab |


In [2]:
!pip install streamlit==1.46.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.8 MB/s eta 0:00:00


In [8]:
!curl http://loca.lt/mytunnelpassword

34.87.77.59

In [54]:
%%writefile main.py
import streamlit as st

st.title("HELLO WORLD")

Overwriting main.py


In [7]:
!streamlit run app.py & npx localtunnel --port 8501



⠙⠹⠸
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.87.77.59:8501

⠼⠴⠦⠧⠇⠏your url is: https://cold-ravens-teach.loca.lt
2025-07-03 03:11:31.218350: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751512291.242322    2553 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751512291.249362    2553 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-03 03:11:31.276283: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the follow

In [6]:
%%writefile app.py
import streamlit as st
import tempfile
import os
import torch
from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface.llms import HuggingFacePipeline
from langchain.memory import ConversationBufferMemory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain.chains import ConversationalRetrievalChain
from langchain_experimental.text_splitter import SemanticChunker
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub

if 'models_loaded' not in st.session_state:
  st.session_state.models_loaded = None

if 'embedding' not in st.session_state:
  st.session_state.embedding = None

if 'llm' not in st.session_state:
  st.session_state.llm = None

#dùng để tránh gọi nhiều lần, đỡ tồn tài nguyên, cache sẵn
#chỉ có tác dụng lên hàm ngay dưới đó
@st.cache_resource
def load_embedding():
  return HuggingFaceEmbeddings(
      model = 'intfloat/multilingual-e5-base'
  )

def load_llm():

  MODEL_NAME = 'lmsys/vicuna-7b-v1.5'

  config_nf4 = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_compute_dtype= torch.float16,
      bnb_4bit_quant_type='nf4',
      bnb_4bit_use_double_quant=True
  )

  model = AutoModelForCausalLM.from_pretrained(
      MODEL_NAME,
      quantization_config = config_nf4,
      low_cpu_mem_usage = True,
  )

  tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

  model_pipeline = pipeline(
      'text-generation',
      model = model,
      max_new_tokens = 512,
      pad_token_id = tokenizer.eos_token_id,
      tokenizer = tokenizer,
      device_map = 'auto'
  )

  llm = HuggingFacePipeline(
      pipeline = model_pipeline
  )

  return llm

def process_pdf(file_uploaded):
  with tempfile.NamedTemporaryFile(delete=False, suffix='.pdf') as tmp_file:
    tmp_file.write(file_uploaded.getvalue())
    tmp_file_path = tmp_file.name

  Loader = PyPDFLoader
  loader = Loader(tmp_file_path)
  document = loader.load()

  semantic_splitter = SemanticChunker(
      embeddings = st.session_state.embedding,
      add_start_index = True,
      buffer_size=1,
      breakpoint_threshold_type="percentile",
      breakpoint_threshold_amount=95,
      min_chunk_size=500
  )

  docs = semantic_splitter.split_documents(document)
  vector_db = Chroma.from_documents(documents = docs, embedding=st.session_state.embedding)
  retriver = vector_db.as_retriever()

  prompt = hub.pull('rlm/rag-prompt')

  def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

  rag_chain = (
      {'context': retriver|format_docs, 'question': RunnablePassthrough()}
      | prompt
      | st.session_state.llm
      | StrOutputParser()
  )

  os.unlink(tmp_file_path)
  return rag_chain, len(docs)


def main():
  st.set_page_config(page_title="PDF RAG Assistance", layout='wide')
  st.title("PDF RAG Assistant")

  st.markdown("""
  **Ứng dụng AI giúp bạn hỏi đáp trực tiếp với nội dung tài liệu PDF bằng tiếng Việt**

  **Cách sử dụng đơn giản:**
  1. **Upload PDF** → Chọn file PDF từ máy tính và nhấn "Xử lý PDF"
  2. **Đặt câu hỏi** → Nhập câu hỏi về nội dung tài liệu và nhận câu trả lời ngay lập tức

  ---
  """)

  if not st.session_state.models_loaded:
    st.info("Loading models")
    st.session_state.embedding = load_embedding()
    st.session_state.llm = load_llm()
    st.session_state.models_loaded = True
    st.success("Model Preparation is Done!")
    st.rerun()

  uploaded_file = st.file_uploader("Upload your PDF file", type='pdf')
  if uploaded_file and st.button("Press to process file"):
    st.session_state.rag_chain, num_chunks = process_pdf(uploaded_file)
    st.success(f"The semantic split on file is done. Number of chunks is {num_chunks}")

  question = st.text_input("Please input a question")
  if question:
    with st.spinner("Answering question..."):
        output = st.session_state.rag_chain.invoke(question)
        answer = output.split("Answer: ")[1] if "Answer: " in output else output
        st.write("The answer is:")
        st.write(answer)



if __name__ == '__main__':
  main()



Overwriting app.py
